In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage import io, transform
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [ ]:
train_data = pd.read_csv("../input/train.csv")
test_data = pd.read_csv("../input/test.csv")
print(train_data.shape,
     test_data.shape)

In [ ]:
x = np.array(train_data.drop(['label'], axis = 1))
y = np.array(train_data['label'])
test_data = np.array(test_data)

x = x.reshape((x.shape[0], 28, 28, 1))
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

x = x/255
test_data = test_data/255
y = to_categorical(y, num_classes = 10)

print(x.shape, y.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.10, shuffle = True)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))
model.summary()

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size = 86),
                              epochs = 30, validation_data = (x_test,y_test),
                              verbose = 1, 
                              callbacks=[learning_rate_reduction])

In [ ]:
eval_res = model.evaluate(x, y, batch_size = 32)
print("Accuracy is: ", eval_res[1])

In [ ]:
pred = model.predict_classes(test_data, batch_size = 64).astype(int)

In [ ]:
ImageId = []
for i in range(1, 28001, 1):
    ImageId.append(i)

In [ ]:
submit = pd.DataFrame({'ImageId': ImageId, 'Label' : pred})
submit.to_csv('submission1.csv', index = False)